Reference -- https://colab.research.google.com/drive/1MvFJpyMStxDlq31wRshWB07nykxoME3-?usp=sharing

## Create a YouTube Video Summarizer Using Whisper and LangChain 
We previously explored the powerful feature of LangChain called chains, which allow for the creation of an end-to-end pipeline for using language models. We learned how chains combine multiple components such as models, prompts, memory, parsing output, and debugging to provide a user-friendly interface. We also discussed the process of designing custom pipelines by inheriting the Chain class and explored the LLMChain as a simple example. This lesson served as a foundation for future lessons, where we will apply these concepts to a hands-on project of summarizing a YouTube video.

During this lesson, we delved into the challenge of summarizing YouTube videos efficiently in the context of the digital age. It will introduce two cutting-edge tools, Whisper and LangChain, that can help tackle this issue. We will discuss the strategies of "stuff," "map-reduce," and "refine" for handling large amounts of text and extracting valuable information. It is possible to effectively extract key takeaways from videos by leveraging Whisper to transcribe YouTube audio files and utilizing LangChain's summarization techniques, including stuff, refine, and map_reduce. We also highlighted the customizability of LangChain, allowing personalized prompts, multilingual summaries, and storage of URLs in a Deep Lake vector store. By implementing these advanced tools, you can save time, enhance knowledge retention, and improve your understanding of various topics. Enjoy the tailored experience of data storage and summarization with LangChain and Whisper.

The following diagram explains what we are going to do in this project.

<img src="../../images/YoutubeVideo.png" width=500 height=100 />

First, we download the youtube video we are interested in and transcribe it using Whisper. Then, we’ll proceed by creating summaries using two different approaches:

1. First we use an existing summarization chain to generate the final summary, which automatically manages embeddings and prompts.

2. Then, we use another approach more step-by-step to generate a final summary formatted in bullet points, consisting in splitting the transcription into chunks, computing their embeddings, and preparing ad-hoc prompts.

In [3]:
# !pip install langchain==0.0.208 deeplake openai==0.27.8 tiktoken scikit-learn deeplake

Additionally, install also the `yt_dlp` and `openai-whisper` packages.

In [7]:
# !pip install -q yt_dlp
# !pip install -q git+https://github.com/openai/whisper.git

Then, we must install the ffmpeg application, which is one of the requirements for the yt_dlp package.

In [4]:
# MacOS (requires https://brew.sh/)
#brew install ffmpeg

# Ubuntu
#sudo apt install ffmpeg

In [5]:
# Load environment variables
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

True

For this experiment, we have selected a video featuring Yann LeCun, a distinguished computer scientist and AI researcher. In this engaging discussion, LeCun delves into the challenges posed by large language models.

The `download_mp4_from_youtube()` function will download the best quality mp4 video file from any YouTube link and save it to the specified path and filename. We just need to copy/paste the selected video’s URL and pass it to mentioned function.

In [6]:
import yt_dlp

def download_mp4_from_youtube(url):
    # Set the options for the download
    filename = 'lecuninterview.mp4'
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]',
        'outtmpl': filename,
        'quiet': True,
    }

    # Download the video file
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        result = ydl.extract_info(url, download=True)

url = "https://www.youtube.com/watch?v=mBjPyte2ZZo"
download_mp4_from_youtube(url)

### Now it’s time for Whisper!

Whisper is a cutting-edge, automatic speech recognition system developed by OpenAI. Boasting state-of-the-art capabilities, Whisper has been trained on an impressive 680,000 hours of multilingual and multitasking supervised data sourced from the web.  This vast and varied dataset enhances the system's robustness, enabling it to handle accents, background noise, and technical language easily. OpenAI has released the models and codes to provide a solid foundation for creating valuable applications harnessing the power of speech recognition.

The whisper package that we installed earlier provides the `.load_model()` method to download the model and transcribe a video file. Multiple different models are available: `tiny`, `base`, `small`, `medium`, and `large`. Each one of them has tradeoffs between accuracy and speed. We will use the `base` model for this tutorial.

In [8]:
import whisper

model = whisper.load_model("base")
result = model.transcribe("lecuninterview.mp4")
print(result['text'])

/Users/smaz-home/anaconda3/lib/python3.8/site-packages/numba/core/types/__init__.py:108: FutureWarning: In the future `np.long` will be defined as the corresponding NumPy scalar.
  long_ = _make_signed(np.long)


AttributeError: module 'numpy' has no attribute 'long'